In [ ]:
df = x.copy()
df.dtypes

In [ ]:
df.columns

In [ ]:
features_to_include = ['party_size','monster_number','monster_total_level','party_total_ac',
                      'party_total_prof_bonus',
                      'party_total_strength', 'party_total_dexterity',
                       'party_total_constitution',
                      'party_total_intelligence',
                      'party_total_wisdom',
                       'party_total_charisma', 'player_monster_ratio','monster_player_ratio', 'Druid','Cleric','Wizard','Rogue','Warlock','Sorcerer',
                       'Blood Hunter','Monk','Bard','Barbarian','Fighter','Paladin','Ranger',
                      'weighted_monster_level']

## Baseline Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming df is your DataFrame

# Prepare your data
X = df[['monster_total_level']]  # Predictor
y = df['party_total_hpratio']  # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Always predicting the mean
mean_train = np.mean(y_train)

# Create an array filled with the mean value that matches the length of the test set
mean_predictions = np.full(shape=y_test.shape, fill_value=mean_train)

# Evaluating the model
mse = mean_squared_error(y_test, mean_predictions)
rmse = np.sqrt(mse)

print("Baseline Model RMSE:", rmse)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming df is your DataFrame

# Prepare your data
X = df['monster_total_level'].values.reshape(-1,1) # Predictor
y = df['party_total_hpratio']  # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=455)

# Train a Linear Regression Model
model = LinearRegression()
model.fit(X_train, y_train)

# Predicting on the test set
y_pred = model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("Baseline Model RMSE:", rmse)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np


# Prepare your data
X = df['weighted_monster_level'].values.reshape(-1,1)  # Predictor

y = df['party_total_hpratio']  # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Linear Regression Model
model = LinearRegression()
model.fit(X_train, y_train)

# Predicting on the test set
y_pred = model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("Baseline Model RMSE:", rmse)


## Random Forest Regressor

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


# Defining the target variable
target = 'party_total_hpratio'

# Preparing the features and target variable
X_rf = df[features_to_include].copy()
y = df[target].copy()

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_rf, y, test_size=0.2, random_state=42)



# Creating the RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Training the model
regressor.fit(X_train, y_train)

# Predicting on the test set
y_pred = regressor.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
rmse
plt.scatter(y_test,y_pred)

## PLA

In [ ]:
import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

X_PLA = df[features_to_include].copy()
Y = df[target]

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X_PLA, Y, random_state=35)

# Initial PLS model fitting with a predefined number of components
pls = PLSRegression(n_components=4)
pls.fit(X_train, y_train)
Y_pred = pls.predict(X_test)

# Using KFold for cross-validation to find the optimal number of components
kf = KFold(n_splits=5, shuffle=True, random_state=42)
max_components = 20
scores = []
for i in range(1, max_components + 1):
    pls = PLSRegression(n_components=i)
    # Make sure to use X_train and y_train for cross-validation
    score = -cross_val_score(pls, X_train, y_train, cv=kf, scoring='neg_mean_squared_error').mean()
    scores.append(score)
    
optimal_components = np.argmin(scores) + 1

# Evaluation metrics for the initial model (consider re-evaluating after selecting the optimal number of components)
print(f"R-squared: {r2_score(y_test, Y_pred)}")
print(f"Mean Squared Error: {mean_squared_error(y_test, Y_pred)}")
print(f"Optimal number of components based on CV: {optimal_components}")



In [ ]:
plt.scatter(y_test,Y_pred)